In [128]:
import hashlib
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES
import os

v_bits=1024
iv_bytes=os.urandom(16)

####### TEST value #####
iv_bytes=('i'*16).encode()
########################

# skey must be 256 bits long
def UAB_E(skey, plain_bytes):
    if len(plain_bytes)%16 != 0:
        print("AES_ERROR: La mida de la cadena a xifrar ha de ser múltiple de 128 bits")
        return None
    else:
        cipher_config = AES.new(skey, AES.MODE_CBC, iv_bytes)
        return cipher_config.encrypt(plain_bytes)

def UAB_E_inv(skey, enc_bytes):
    cipher = AES.new(skey, AES.MODE_CBC, iv_bytes)
    return cipher.decrypt(enc_bytes)

def UAB_h(m):
    return hashlib.sha256(m.encode('utf-8')).digest()

def UAB_xor(a, b):
    v_len_bytes = v_bits/8
    a_ba=bytearray(v_len_bytes-len(a))+a
    b_ba=bytearray(v_len_bytes-len(b))+b
    return bytes(b_a ^^ b_b for b_a, b_b in zip(a_ba, b_ba))

#Returns a tuple with the (n, e) values of a new public key
def UAB_generate_RSA_public_key(nBits):
    keys = RSA.generate(bits=nBits)
    pk = keys.publickey()
    return (pk.n, pk.e)

#Returns a tuple ((n, e), (n, d)) containing a public key tuple in the 1st position and a private key tuple in the 2nd position.
def UAB_generate_RSA_key_pair(nBits):
    keys = RSA.generate(bits=nBits)
    pk = keys.publickey()
    pk_vars = (pk.n, pk.e)
    sk_vars =  (keys.n, keys.d)
    return(pk_vars, sk_vars)


In [129]:
# EXERCISE 1: Implement auxiliary functions.
# Implement the following auxiliary functions that we will use to generate and verify ring signatures.

# PK = public key (n,e)
# x = bytes object to cypher using RSA public key PK
# returns a bytes object
def UAB_f(PK,x):
   
    n, e = PK
    #Ho convertim a enter
    enter = int.from_bytes(x, 'big')
    q, r = divmod(enter, n)
    if (q+1)*n <= 2^v_bits:
        #Xifrem amb RSA --> C = X^e
        c = pow(r,e,n)
        res = q*n + c
    else:
        res = enter
    #Ho convertim a bytes
    res_bytes = int(res).to_bytes((n.bit_length()+7)//8, 'big')
    
    
    return res_bytes

# SK = private key (n,d)
# y = bytes object to decypher using RSA private key SK
# returns a bytes object
def UAB_f_inv(SK,y):
    
    n, d = SK
    #Ho convertim a enter
    enter = int.from_bytes(y, 'big')
    q, r = divmod(enter, n)
    if (q+1)*n <= 2^v_bits:
        #Desxifrem amb RSA --> m = y^d
        m = pow(r,d,n)
        res = q*n + m
    else:
        res = enter
    #Ho convertim a bytes
    res_bytes = int(res).to_bytes((n.bit_length()+7)//8, 'big')
    
    
    return res_bytes

# TESTS

x_test = ('x'*int(v_bits/8)).encode()
Pub_key,Priv_key=UAB_generate_RSA_key_pair(1024)
encrypted=UAB_f(Pub_key,x_test)
decrypted=UAB_f_inv(Priv_key,encrypted)
decrypted_string=decrypted.decode()
if decrypted_string == 'x'*int(v_bits/8):
    print("Correct")
else:
    print("Error")

Correct


In [130]:
# EXERCISE 2: Setting the scenario.
# Generate all the variables required to sign the message with a ring signature.

#### IMPLEMENTATION GOES HERE ####

# Length of the ring.
LENGTH= len(ys)

# Signer position (considering the first position as position=0)
position= 1

# Public key (pkSigner) and private key (skSigner) of the signer.

pkSigner,skSigner= UAB_generate_RSA_key_pair(1024)


# List with the public keys of the group. This list has to include the public key of the signer in position='position'
publicKeysGroup= []

for i in range(LENGTH):
    if i == position:
        #Per la posició donada fiquem la nostra clau pública
        publicKeysGroup.append(pkSigner)
    else:
        public, _ = UAB_generate_RSA_key_pair(1024)
        publicKeysGroup.append(public)
        

# xs and ys of the other members of the group. IMPORTANT: Assign None in the position of the signer.
# All xs must be v_bits long

xs0=randint(0, LENGTH - 1)
xs2=randint(0, LENGTH - 1)
xs3=randint(0, LENGTH - 1)
xs4=randint(0, LENGTH - 1)

xs= [xs0,None,xs2,xs3,xs4]

x_bytes0 = int(xs0).to_bytes(v_bits // 8, 'big')
x_bytes2 = int(xs2).to_bytes(v_bits // 8, 'big')
x_bytes3 = int(xs3).to_bytes(v_bits // 8, 'big')
x_bytes4 = int(xs4).to_bytes(v_bits // 8, 'big')

ys0=UAB_f(publicKeysGroup[0],x_bytes0)
ys2=UAB_f(publicKeysGroup[2],x_bytes2)
ys3=UAB_f(publicKeysGroup[3],x_bytes3)
ys4=UAB_f(publicKeysGroup[4],x_bytes4)
ys= [ys0,None,ys2,ys3,ys4]




In [131]:
# EXERCISE 3: Implement a function to obtain the y value for the signer by solving equation
#             C_k_v(y1, y2, ..., yr) = v.
#
# Function UAB_solve_C.
# * Parameter k: key used in the symmetric crypto system
# * Parameter v: initialization vector
# * Parameter ys: f(x) for all 'x' in 'xs' list. If signer user is i, then y[i] = None.
#    E.g. if in a 6 user array, signer is in position 3, then ys = [y0,y1,y2,None,y4,y5]
# * Returns: y value of the signer as a bytes object

def UAB_solve_C(k,v,ys):
    
    y = v
    
    
    #SENTIT HORARI
    for i in range(0,position):
        if i != position:
            y = UAB_xor(y, ys[i])
            y = UAB_E(k, y)
    #y = UAB_E(k,y)        
        
    y2 = v
    #SENTIT ANTI-HORARI
    for i in range(len(ys)-1, position , -1):
        if i != position:
            y2 = UAB_E_inv(k, y2)
            y2 = UAB_xor(y2, ys[i])
    y2 = UAB_E_inv(k, y2)      
            
    res = UAB_xor(y, y2)
    
    return res

# TESTS
# IMPORTANT: the following must be run with the iv_bytes test value defined above in the 2nd cell.

#position=1
#k=UAB_h("test key")
#v=('v'*int(v_bits/8)).encode()

#ys_t=['f9361a985b27a8aa48a666bf0f57d37c45704a3cd7c01a8c8cd36db3ee65377a85f83393940e81857fbb3e5916f1599ce1f481f046eb215254e9ad2ee3c326047100564136d78942a1984556889c67e0cdef8067b82d3e14841d592383b450f7516f9e1706dbcab969143e6cbbc35b04561a8733f20efedcaeed4f7462908033', '888b3b3ee6999f02b96f4c6f2a0764a6b08678813df4c71aa9de6fad2c975d2c67653da0912c658e28dc16368a3175bb4841dec236919a74216db1d7811b6203f74c0fd8850c3eb9a505cd7a91ef78cad2e07675552e0f79b0b13108ba5eba126127135ba971b4be6b39482d7fb7441fda8ffd888c699144d7baccd95c337429', '49c9375e247d4a3eb49f40911745d701a1722f03db3c94f6d36d6710a988c791a2a7476c6495a2fd502c68ebc44edb3abd5f5bbfe321f80d76b6b3e39317e6be94009e06645fc082349f4954e93866fb89746a20c6d9d60913157d5a7a9960acbc1af34cbe786fee27746edf526def375a2cccc8599a81cb6bbf3a6c04f462c7', '65cd1df2982e9d14a5e9cbc69f6584c070340de16f29c6db1ddafb9b4ab6d3dc29c07b36ae0acf691efdcdaf280e961f22761650c7f08324537174ae93cc7fa780e0a319737b135f7278ed1da55e27c83db7f9661d8f85b473061c3d86159085b4e286903e15e0cb8d3d33211beff50dc3a500fb6b0a62c38fb0db27a1b91d6a']
#ys_test=[bytearray.fromhex(y) for y in ys_t]

#y=UAB_solve_C(k,v,ys_test)

#if y == ys_test[position]:
#    print ("Correct")
#else:
#    print ("Error")
    
#print (y.hex())

# OUTPUT:

# Correct
# 888b3b3ee6999f02b96f4c6f2a0764a6b08678813df4c71aa9de6fad2c975d2c67653da0912c658e28dc16368a3175bb4841dec236919a74216db1d7811b6203f74c0fd8850c3eb9a505cd7a91ef78cad2e07675552e0f79b0b13108ba5eba126127135ba971b4be6b39482d7fb7441fda8ffd888c699144d7baccd95c337429


In [167]:
# EXERCISE 4: Implement the UAB_sign_ring function.
#
# Function UAB_sign_ring
# * Parameter m: message to sign (text string).
# * Parameter v: initialization vector.
# * Returns: ring signature sigma. Sigma is a tuple with three elements:
#    1. List of public keys of the group members.
#    2. Initialization vector v.
#    3. List of the x values of all the members.

def UAB_sign_ring(m, v):
    
      
    vectorIni = v
    k = UAB_h(m)
         
    #Troba la y amb la funció anterior
    ys[position] = UAB_solve_C(k,v,ys)
    #Troba la x relacionada amb la y anterior
    xspp = UAB_f_inv(skSigner,ys[position])
    #x_bytespp = int(xspp).to_bytes(v_bits // 8, 'big')
    xs[position] = xspp
   
    
    
    return (publicKeysGroup, vectorIni, xs)

In [168]:
# EXERCISE 5: Implement the UAB_verify_ring_signature function
#
# Function UAB_verify_ring_signature.
# * Parameter m: message that has been signed (text string).
# * Parameter sigma: ring signature. This is a tuple with three elements: 
#    1. List of public keys of the group members.
#    2. Initialization vector v.
#    3. List of the x values of all the members.
# * Returns: a boolean value indicating if sigma is a valid ring signature for the message m.

def UAB_verify_ring_signature(m, sigma):
    res = None
    
    
    pks, vIni, xss = sigma
    yss = [None] * len(publicKeysGroup)
    
    for i in range(len(xss)):
        yss[i] = UAB_f(publicKeysGroup[i], xss[i])
     
    k = UAB_h(m)
    
    z = vIni

    for y in yss:
    
        z = UAB_h(str(k) + str(y) + str(z))

        
    
    return vIni == z

In [169]:
# TESTS

res_final=True
for i in range(1):

    m_i=str(i)*10
    
    #Initialization vector
    vector_ini = os.urandom(v_bits/8)
    
    sigma_i = UAB_sign_ring(m_i, vector_ini)
    
    if not UAB_verify_ring_signature(m_i, sigma_i):
        print("FALSE")
        res_final=False
    else:
        print("TRUE")
    

print("FINAL RESULT = ",res_final)


TypeError: cannot convert 'int' object to bytes